In [13]:
import snap
import re

In [14]:
comment_file = 'reddit_comments_jan2012.txt'

In [15]:
int_attrs = ['score', 'gilded', 'created_utc']
str_attrs = ['subreddit', 'comment_id']
uunet = snap.TNEANet.New()
uunet_nodelete = snap.TNEANet.New() # No node for [deleted] placeholder, no edges for [deleted] or [removed] comments
uu_simple = snap.TNGraph.New() # Copy without attributes or multi-edges
uu_simple_nodelete = snap.TNGraph.New() # Copy without attributes or multi-edges. Edges ARE added if comment text
                                        # is [removed] or [deleted], unlike uunet_nodelete

for ia in int_attrs:
    uunet.AddIntAttrE(ia)
    uunet_nodelete.AddIntAttrE(ia)
for sa in str_attrs:
    uunet.AddStrAttrE(sa)
    uunet_nodelete.AddStrAttrE(sa)
    
uunet.AddStrAttrN('username')
uunet_nodelete.AddStrAttrN('username')

In [17]:
comments = open(comment_file)
header = comments.readline()[1:].rstrip('\n').split('\t')
fields = {name: i for (i, name) in enumerate(header)}
usernames_to_nids = {}

for (i, com) in enumerate(comments):
    entries = com.rstrip('\n').split('\t')
    subreddit = entries[fields['Subreddit']]
    commenter = entries[fields['Commenter']] # standardized to lowercase
    commentee = entries[fields['Commentee']] # ditto
    comment_id = entries[fields['Comment_ID']]
    
    score = int(entries[fields['Score']])
    gilded = int(entries[fields['Gilded']])
    created_utc = int(entries[fields['Created_UTC']])
    
    text_deleted = int(entries[fields['Text_Deleted']])
    
    # create nodes for commenter and commentee if they don't exist
    for user in (commenter, commentee):
        if user not in usernames_to_nids:
            nid = usernames_to_nids[user] = uunet.AddNode()
            uunet.AddStrAttrDatN(nid, user, 'username')
            uu_simple.AddNode(nid)
            if user != '[deleted]':
                uunet_nodelete.AddNode(nid)
                uunet_nodelete.AddStrAttrDatN(nid, user, 'username')
                uu_simple_nodelete.AddNode(nid)
    
    commenter_nid = usernames_to_nids[commenter]
    commentee_nid = usernames_to_nids[commentee]
    
    target_nets = [uunet]
    if commenter != '[deleted]' and commentee != '[deleted]' and not text_deleted:
        target_nets.append(uunet_nodelete)
    
    # Add comment edge with attributes to TNEANet(s)
    for net in target_nets:
        eid = net.AddEdge(commenter_nid, commentee_nid)
        
        net.AddIntAttrDatE(eid, score, 'score')
        net.AddIntAttrDatE(eid, gilded, 'gilded')
        net.AddIntAttrDatE(eid, created_utc, 'created_utc')
        net.AddStrAttrDatE(eid, subreddit, 'subreddit')
        net.AddStrAttrDatE(eid, comment_id, 'comment_id')
        
    # Add interaction edge (if not already present) to TNGraph(s)
    uu_simple.AddEdge(commenter_nid, commentee_nid)
    if commenter != '[deleted]' and commentee != '[deleted]':
        uu_simple_nodelete.AddEdge(commenter_nid, commentee_nid)
    
    # Show progress
    if i % 100000 == 0 and i > 0:
        print(i)

# Save networks        
out_multi_delete = snap.TFOut('usernets/user_tneanet.graph')
out_multi_nodelete = snap.TFOut('usernets/user_tneanet_nodelete.graph')
out_simple_delete = snap.TFOut('usernets/user_tngraph.graph')
out_simple_nodelete = snap.TFOut('usernets/user_tngraph_nodelete.graph')

uunet.Save(out_multi_delete)
out_multi_delete.Flush()
uunet_nodelete.Save(out_multi_nodelete)
out_multi_nodelete.Flush()
uu_simple.Save(out_simple_delete)
out_simple_delete.Flush()
uu_simple_nodelete.Save(out_simple_nodelete)
out_simple_nodelete.Flush()

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [21]:
# Save table mapping usernames to NIDs
users_to_nids_file = open('usernets/usernames_to_nids.tsv', 'w')
users_to_nids_file.write('#{:20}\tNode_ID\n'.format('Username'))
for (username, nid) in sorted(usernames_to_nids.iteritems()):
    users_to_nids_file.write('{:20}\t{}\n'.format(username, nid))
users_to_nids_file.close()